In [305]:
import pandas as pd
df = pd.read_csv('houses_train.csv')

In [306]:
df.head()

,Unnamed: 0,price,condition,district,max_floor,street,num_rooms,region,area,url,num_bathrooms,building_type,floor,ceiling_height
0,4598,100000.0,newly repaired,Arabkir,6,Kievyan St,3,Yerevan,96.0,http://www.myrealty.am/en/item/26229/3-senyaka...,1,stone,4,3.0
1,5940,52000.0,good,Arabkir,14,Mamikoniants St,3,Yerevan,78.0,http://www.myrealty.am/en/item/32897/3-senyaka...,1,panel,10,2.8
2,2302,52000.0,newly repaired,Qanaqer-Zeytun,9,M. Melikyan St,3,Yerevan,97.0,http://www.myrealty.am/en/item/1459/apartment-...,1,panel,1,2.8
3,5628,130000.0,good,Center,4,Spendiaryan St,3,Yerevan,80.0,http://www.myrealty.am/en/item/2099/3-senyakan...,1,stone,2,3.2
4,760,81600.0,zero condition,Center,9,Ler. Kamsar St,3,Yerevan,107.0,http://www.myrealty.am/en/item/22722/3-senyaka...,1,monolit,9,3.0


In [397]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pickle
from sklearn.pipeline import make_pipeline

# Preprocessing

In [308]:
X = df.drop(['Unnamed: 0', 'price', 'region', 'url'], axis=1)
y = df['price']

In [398]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=11)

In [343]:
cat_columns = X_train.select_dtypes(include='object').columns

In [344]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[cat_columns])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [345]:
X_train[ohe.get_feature_names_out()] = ohe.transform(X_train[cat_columns])
X_val[ohe.get_feature_names_out()] = ohe.transform(X_val[cat_columns])

/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/2687537608.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[ohe.get_feature_names_out()] = ohe.transform(X_train[cat_columns])
/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/2687537608.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[ohe.get_feature_names_out()] = ohe.transform(X_train[cat_columns])
/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/2687537608.py:1: PerformanceWarning: DataFrame is highly fragme

In [348]:
X_train.drop(cat_columns, axis=1, inplace=True)
X_val.drop(cat_columns, axis=1, inplace=True)

In [351]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

# KNN

In [358]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
cross_val_score(knn, X_train, y_train).mean()

0.7178233576811548

In [367]:
gs = GridSearchCV(knn, {'n_neighbors': range(1, 15), 'weights':['uniform', 'distance']})
gs.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 15),
                         'weights': ['uniform', 'distance']})

In [368]:
gs.best_estimator_

KNeighborsRegressor(n_neighbors=12, weights='distance')

In [372]:
cross_val_score(gs.best_estimator_, X_train, y_train).mean()

0.7489609223746168

# Ridge

In [373]:
from sklearn.linear_model import Ridge

In [378]:
r = Ridge()
gs_r = GridSearchCV(r, {'alpha': [1.1, 1.3, 1.5, 2, 2.5]})
gs_r.fit(X_train, y_train)

GridSearchCV(estimator=Ridge(), param_grid={'alpha': [1.1, 1.3, 1.5, 2, 2.5]})

In [380]:
cross_val_score(gs_r.best_estimator_, X_train, y_train).mean()

0.7859746633460057

In [381]:
final_cls = gs_r.best_estimator_
final_cls.fit(X_train, y_train)

Ridge(alpha=1.5)

In [382]:
final_cls.score(X_val, y_val)

0.7748794035938595

In [384]:
from sklearn.metrics import mean_squared_error
mean_squared_error(final_cls.predict(X_val), y_val, squared=False)

23578.28623202976

In [386]:
pickle.dump(ohe, open('ohe.sav', 'wb'))
pickle.dump(scaler, open('scaler.sav', 'wb'))
pickle.dump(final_cls, open('model.sav', 'wb'))

In [387]:
def final_predict(final_test_df):
    ohe = pickle.load(open('ohe.sav', 'rb'))
    scaler = pickle.load(open('scaler.sav', 'rb'))
    final_cls = pickle.load(open('model.sav', 'rb'))

    final_test_df = final_test_df.drop(['Unnamed: 0', 'region', 'url'], axis=1)
    cat_columns = final_test_df.select_dtypes(include='object').columns
    final_test_df[ohe.get_feature_names_out()] = ohe.transform(final_test_df[cat_columns])
    final_test_df.drop(cat_columns, axis=1, inplace=True)
    final_test_df = scaler.transform(final_test_df)
    return final_cls.predict(final_test_df)

In [393]:
df_test = pd.read_csv('houses_test.csv')
y_test = df_test['price']
df_test = df_test.drop('price', axis=1)
y_pred = final_predict(df_test)

/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/191381788.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_test_df[ohe.get_feature_names_out()] = ohe.transform(final_test_df[cat_columns])
/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/191381788.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_test_df[ohe.get_feature_names_out()] = ohe.transform(final_test_df[cat_columns])
/var/folders/sy/y3jcsc0x39zb38_bqw16xm8m0000gr/T/ipykernel_71437/191381788.py:8: PerformanceWarning: DataF

In [396]:
len(df_test)

1234

In [395]:
mean_squared_error(y_pred, y_test, squared=False)

24614.830157679593

# Pipeline

In [400]:
X_train_num = X_train.select_dtypes(exclude='object')

In [403]:
pip = make_pipeline(our_ohe(), MinMaxScaler(), Ridge())
pip.fit(X_train_num, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('ridge', Ridge())])

In [405]:
pip.predict(X_train_num)

array([ 94367.58054172,  85239.56262302,  70550.73541973, ...,
       109291.55800921,  44702.26972997,  65972.10556372])

In [404]:
pip.score(X_train_num, y_train)

0.49994794729170033

In [ ]:
class our_ohe:
    def __init__():
        ohe = OneHotEncoder()
    
    def fit(X_train, y_train):
        # call ohe.fit() and do your preprocessing
        
    def transform(X_test):
        # call ohe.transform() and do your postprocessing

# how to make your model better
1. use polynomial features
2. use lasso to select features
3. group less frequent categories into 1 before ohe
4. use different scalings
5. use different approximating functions
6. use floor and max_floor to get new feature
7. use feature floor==max_floor